In [3]:
import sys
!{sys.executable} -m pip install python-dotenv

'C:\Users\fatim\Data' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import os
import requests
import json
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Load the API key from your .env file
load_dotenv()
API_KEY = os.getenv('GOOGLE_BOOKS_API_KEY')

def extract_full_book_data(query):
    """
    Performs a search and returns all possible fields using projection=full.
    """
    url = "https://www.googleapis.com/books/v1/volumes"
    
    # projection='full' is the key to getting maximum data
    params = {
        'q': query,
        'key': API_KEY,
        'projection': 'full',  # Unlocks high-res images, dimensions, etc.
        'maxResults': 5        # Limit to 5 for this example
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        books_extracted = []
        
        for item in data.get('items', []):
            info = item.get('volumeInfo', {})
            sale = item.get('saleInfo', {})
            access = item.get('accessInfo', {})
            
            # Map out as much data as possible
            book_record = {
                "id": item.get('id'),
                "title": info.get('title'),
                "subtitle": info.get('subtitle'),
                "authors": info.get('authors', []),
                "publisher": info.get('publisher'),
                "published_date": info.get('publishedDate'),
                "description": info.get('description'),
                "industry_identifiers": info.get('industryIdentifiers', []), # ISBN_10, ISBN_13
                "page_count": info.get('pageCount'),
                "categories": info.get('categories', []),
                "average_rating": info.get('averageRating'),
                "ratings_count": info.get('ratingsCount'),
                "maturity_rating": info.get('maturityRating'),
                "dimensions": info.get('dimensions', {}), # height, width, thickness (FULL only)
                "main_category": info.get('mainCategory'),
                "image_links": info.get('imageLinks', {}), # thumb, small, medium, large, extraLarge
                "language": info.get('language'),
                "preview_link": info.get('previewLink'),
                "info_link": info.get('infoLink'),
                "canonical_link": info.get('canonicalVolumeLink'),
                "sale_info": {
                    "country": sale.get('country'),
                    "saleability": sale.get('saleability'),
                    "is_ebook": sale.get('isEbook'),
                    "list_price": sale.get('listPrice', {}),
                    "retail_price": sale.get('retailPrice', {}),
                    "buy_link": sale.get('buyLink')
                },
                "access_info": {
                    "viewability": access.get('viewability'),
                    "web_reader_link": access.get('webReaderLink'),
                    "pdf_available": access.get('pdf', {}).get('isAvailable'),
                    "epub_available": access.get('epub', {}).get('isAvailable')
                }
            }
            books_extracted.append(book_record)
            
        return books_extracted

    except requests.exceptions.RequestException as e:
        print(f"Error connecting to Google Books API: {e}")
        return None

if __name__ == "__main__":
    search_term = "Data Science with Python"
    results = extract_full_book_data(search_term)
    
    if results:
        # Save the full data to a JSON file for inspection
        with open('full_book_data.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        print(f"Successfully extracted data for {len(results)} books.")
        print("Check 'full_book_data.json' to see all the fields extracted.")